In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import lightgbm as lgb

In [3]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [4]:
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
	sys.path.insert(0, src_path)

from data_loader import load_data
from features import add_time_features, create_target

In [5]:
df = load_data("../data/raw/online_retail_II.csv")

c:\Users\anasm\OneDrive\Desktop\deeplearning\Feature Drift Detection\src\data_loader.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])


In [6]:
X = add_time_features(df)

In [7]:
y = create_target(df , 0.95)

In [8]:
train_mask = df["InvoiceDate"] <= "2010-06-30"
val_mask   = (df["InvoiceDate"] > "2010-06-30") & (df["InvoiceDate"] <= "2010-09-30")
test_mask  = df["InvoiceDate"] > "2010-09-30"


In [9]:
X = X.drop('InvoiceDate', axis=1)
X_train, y_train = X[train_mask], y[train_mask]
X_val,   y_val   = X[val_mask], y[val_mask]
X_test,  y_test  = X[test_mask], y[test_mask]

In [10]:
import optuna
import sklearn.metrics

c:\Users\anasm\miniconda3\envs\torch-ds\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
CATEGORICAL_FEATURES = [
    'Country',
    'StockCode'
]

In [12]:
# List of columns identified in the error
# Convert to category type
for col in CATEGORICAL_FEATURES:
    X_train[col] = X_train[col].astype('category')
    X_val[col] = X_val[col].astype('category')

C:\Users\anasm\AppData\Local\Temp\ipykernel_26324\4100065008.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col] = X_train[col].astype('category')
C:\Users\anasm\AppData\Local\Temp\ipykernel_26324\4100065008.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[col] = X_val[col].astype('category')
C:\Users\anasm\AppData\Local\Temp\ipykernel_26324\4100065008.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [15]:
# Replace StockCode with its frequency count
enc = X_train['StockCode'].value_counts()
X_train['StockCode_Freq'] = X_train['StockCode'].map(enc)
X_val['StockCode_Freq'] = X_val['StockCode'].map(enc).fillna(0)

# Drop the original high-cardinality column
X_train.drop('StockCode', axis=1, inplace=True)
X_val.drop('StockCode', axis=1, inplace=True)

C:\Users\anasm\AppData\Local\Temp\ipykernel_26324\1472325292.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['StockCode_Freq'] = X_train['StockCode'].map(enc)
C:\Users\anasm\AppData\Local\Temp\ipykernel_26324\1472325292.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val['StockCode_Freq'] = X_val['StockCode'].map(enc).fillna(0)
C:\Users\anasm\AppData\Local\Temp\ipykernel_26324\1472325292.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See t

In [18]:
X_val.info()

<class 'pandas.core.frame.DataFrame'>
Index: 108572 entries, 255103 to 363674
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   Quantity        108572 non-null  int64   
 1   Price           108572 non-null  float64 
 2   Country         108572 non-null  category
 3   Year            108572 non-null  int32   
 4   Month           108572 non-null  int32   
 5   Day             108572 non-null  int32   
 6   Hour            108572 non-null  int32   
 7   min             108572 non-null  int32   
 8   sec             108572 non-null  int32   
 9   StockCode_Freq  108572 non-null  float64 
dtypes: category(1), float64(2), int32(6), int64(1)
memory usage: 5.9 MB


In [19]:
CATEGORICAL_FEATURES = [
    'Country'
]

In [22]:
# 1. Create Datasets ONCE (Outside the objective function)

dtrain = lgb.Dataset(X_train, label=y_train, categorical_feature=CATEGORICAL_FEATURES)
dvalid = lgb.Dataset(X_val, label=y_val, categorical_feature=CATEGORICAL_FEATURES, reference=dtrain)


def objective(trial):
    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "feature_pre_filter": False,
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True), # Added this!
        "device": "gpu"
    }

    # 2. Add Early Stopping to save time and prevent overfitting
    gbm = lgb.train(
        param, 
        dtrain, 
        valid_sets=[dvalid],
        num_boost_round=1000,
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(period=0) # Keeps the console clean
        ]
    )
    
    preds = gbm.predict(X_val)
    pred_labels = (preds >= 0.5).astype(int)
    accuracy = sklearn.metrics.accuracy_score(y_val, pred_labels)
    return accuracy

In [24]:
study = optuna.create_study(direction="maximize")
study.optimize(objective , n_trials=10)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2026-02-04 18:06:26,015] A new study created in memory with name: no-name-01d6164f-42f1-4fda-973f-d7ac2988337e


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's binary_logloss: 0.0986612


[I 2026-02-04 18:07:35,398] Trial 0 finished with value: 0.9567383855874443 and parameters: {'lambda_l1': 0.028652102694449143, 'lambda_l2': 0.09258469186474885, 'num_leaves': 190, 'feature_fraction': 0.4210056212040654, 'bagging_fraction': 0.9026468526533117, 'bagging_freq': 1, 'min_child_samples': 80, 'learning_rate': 0.0015620812011877616}. Best is trial 0 with value: 0.9567383855874443.


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's binary_logloss: 0.00983319


[I 2026-02-04 18:08:09,123] Trial 1 finished with value: 0.9997236856648123 and parameters: {'lambda_l1': 7.843519391526596e-05, 'lambda_l2': 2.1027075111341058e-07, 'num_leaves': 77, 'feature_fraction': 0.9099808527227218, 'bagging_fraction': 0.721605124556153, 'bagging_freq': 4, 'min_child_samples': 9, 'learning_rate': 0.0027747038156198103}. Best is trial 1 with value: 0.9997236856648123.


...

Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's binary_logloss: 0.0451498


[I 2026-02-04 18:11:04,693] Trial 8 finished with value: 0.9944737132962458 and parameters: {'lambda_l1': 1.4964266481897632e-05, 'lambda_l2': 1.2477037495140549e-08, 'num_leaves': 47, 'feature_fraction': 0.654679541985153, 'bagging_fraction': 0.7188634173561783, 'bagging_freq': 3, 'min_child_samples': 50, 'learning_rate': 0.0016798458813791464}. Best is trial 3 with value: 0.9997973694875291.


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's binary_logloss: 0.0330936


[I 2026-02-04 18:11:17,678] Trial 9 finished with value: 0.9974671185941126 and parameters: {'lambda_l1': 4.724690987674201e-07, 'lambda_l2': 4.273210775541312, 'num_leaves': 14, 'feature_fraction': 0.9718830065815939, 'bagging_fraction': 0.8704031354900577, 'bagging_freq': 5, 'min_child_samples': 6, 'learning_rate': 0.0015415724043577848}. Best is trial 3 with value: 0.9997973694875291.


Number of finished trials: 10
Best trial:
  Value: 0.9997973694875291
  Params: 
    lambda_l1: 0.0004994201498133918
    lambda_l2: 0.09874754888296922
    num_leaves: 25
    feature_fraction: 0.6197708124969349
    bagging_fraction: 0.46957015107627764
    bagging_freq: 6
    min_child_samples: 52
    learning_rate: 0.021149802451186903


In [27]:
from pathlib import Path

Path("../models").mkdir(parents=True, exist_ok=True)



In [28]:
best_params = study.best_params
best_params.update({
    "objective": "binary",
    "metric": "binary_logloss",
    "verbose": -1,
    "boosting_type": "gbdt",
    "device": "gpu",
})

dtrain = lgb.Dataset(X_train, label=y_train, categorical_feature=CATEGORICAL_FEATURES)
gbm = lgb.train(best_params, dtrain)

gbm.save_model("../models/lgbm_optuna.txt")
